In [9]:
from pyDACP import core, chebyshev
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
from scipy.linalg import eigh, eig
from scipy.sparse import eye, diags
from itertools import groupby

In [12]:
N = 100
deg_tresh = 1e-12
MAX_ITER = 100

b = np.concatenate((np.ones(int(N/4))*-1, np.ones(int(N/2))*0.1, np.ones(int(N/4))*1))
H = diags(b, offsets=0)


def dacp_deg_solver(H):
    random_vectors = 2
    dacp=core.DACP_reduction(H, a=0.2, eps=0.05, bounds=None, sampling_subspace=1.5, random_vectors=random_vectors)
    d_KPM = dacp.estimate_subspace_dimenstion()
    n_KPM = int(np.abs((d_KPM*dacp.sampling_subspace - 1)/2))

    ham_red=dacp.get_subspace_matrix()
    d_red = ham_red.shape[0]

    eigvalues = eigh(ham_red, eigvals_only=True)
    deg_n = np.abs(np.diff(eigvalues)) < deg_tresh
    max_deg = np.max([(x[0], len(list(x[1]))) for x in groupby(deg_n) if x[0]==True]) + 1
    
    if max_deg > 1:
        
        if d_KPM > d_red:
            d_deg = int(1.5*(d_KPM-d_red))
            n_deg = int(np.abs((d_deg*dacp.sampling_subspace - 1)/2))
            random_vector_guess = n_deg
            print('n KPM', n_KPM)
            print('n_deg', n_deg)
            
        else: 
            random_vector_guess = 3

        for i in range(MAX_ITER):
            dacp=core.DACP_reduction(H, a=0.2, eps=0.05, bounds=None, sampling_subspace=1.5, random_vectors=random_vector_guess)
        
            print('Random Vectors Guess', random_vector_guess)

            ham_red=dacp.get_subspace_matrix()
            eigvalues = eigh(ham_red, eigvals_only=True)
            deg_n = np.abs(np.diff(eigvalues)) < deg_tresh
            max_deg = np.max([(x[0], len(list(x[1]))) for x in groupby(deg_n) if x[0]==True]) + 1
            print('max_deg', max_deg)
            if max_deg < random_vector_guess:
                break
            else:
                random_vector_guess = random_vector_guess + 1
                
    return ham_red

In [13]:
dacp_deg_solver(H)

n KPM 31
n_deg 45
Random Vectors Guess 45
max_deg 45
Random Vectors Guess 46
max_deg 46
Random Vectors Guess 47
max_deg 47
Random Vectors Guess 48
max_deg 48
Random Vectors Guess 49
max_deg 49
Random Vectors Guess 50
max_deg 50
Random Vectors Guess 51
max_deg 50


array([[ 1.00000000e-01+3.48462578e-16j, -1.05666344e-15+2.12037419e-15j,
        -1.61589492e-15-3.38098961e-15j, ...,
        -3.00893208e-16-6.96599896e-17j, -1.04672943e-15-1.01830301e-16j,
        -2.18280391e-16+2.29125800e-16j],
       [-3.02882719e-15-3.30464822e-16j,  1.00000000e-01-2.81892565e-16j,
         1.18178037e-15-1.39124823e-15j, ...,
         6.10676874e-17+9.74426703e-18j,  1.60717725e-16+1.29900973e-17j,
         2.64545330e-17-1.25089826e-16j],
       [-1.57859836e-16+2.13468566e-15j, -9.45424294e-17+6.33174069e-16j,
         1.00000000e-01+1.05818132e-16j, ...,
        -2.30935063e-17+1.22514845e-17j,  8.94602318e-17+2.57860546e-16j,
         1.40763323e-16+9.17506088e-17j],
       ...,
       [-4.02455846e-16-2.22044605e-16j,  4.16333634e-17+4.16333634e-17j,
        -1.38777878e-16+0.00000000e+00j, ...,
         1.00000000e-01+1.62630326e-19j,  3.39788961e-16+2.60208521e-18j,
         3.24718551e-17+8.67361738e-19j],
       [-1.33226763e-15+5.55111512e-17j,  4.